# Reinforcement Learning based Virtual Screening
Protein-Ligan pose prediction using RL

In [1]:
from rlvs.molecule_world.env import Env
from rlvs.agents.ddpg_agent import DDPGAgent3D
from matplotlib import pyplot as plt
import numpy as np

### Scoring
RMSD 

In [ ]:
env = Env()

In [ ]:
env.reset()
print(env._complex.rmsd)
print(env._complex.score())

### Protein and Ligand
Protein and ligand layout in space, without any translation

In [3]:
from rlvs.molecule_world.datasets import DataStore
DataStore.init()
DataStore.load()

In [5]:
min_x, min_y, min_z, max_x, max_y, max_z = [None]*6
for protein, ligand in DataStore.DATA:
    _min_x, _min_y, _min_z = protein.coords[:,0].min(), protein.coords[:,1].min(), protein.coords[:,2].min()
    _max_x, _max_y, _max_z = protein.coords[:,0].max(), protein.coords[:,1].max(), protein.coords[:,2].max()
    min_x = _min_x if min_x is None or (_min_x < min_x) else min_x
    min_y = _min_y if min_y is None or (_min_y < min_y) else min_y
    min_z = _min_z if min_z is None or (_min_y < min_z) else min_z
    
    max_x = _max_x if max_x is None or (_max_x > max_x) else max_x
    max_y = _max_y if max_y is None or (_max_y > max_y) else max_y
    max_z = _max_z if max_z is None or (_max_y > max_z) else max_z
max(max_x - min_x, max_y - min_y, max_z - min_z)

62.24

In [ ]:
print(env._complex.box_size)
env._complex.plot_tensor(feature_axis=-1, protein_alpha=0.4)

Protein and ligand layout in space, with randomly generated pose

In [ ]:
xx = env.action_space.sample()
state, reward, t = env.step(xx)
env._complex.plot_tensor(feature_axis=-1, protein_alpha=0.4)
print(reward, xx, env.action_space.action_bounds.shape)

In [ ]:
xx = env.action_space.sample()
state, reward, t = env.step(xx)
env._complex.plot_tensor(feature_axis=-1, protein_alpha=0.4)
print(reward, xx, env.action_space.action_bounds.shape)

### RL Agent
#### Training
Create environment and feed it to the RL agent

In [1]:
from rlvs.molecule_world.env import Env
from rlvs.agents.ddpg_agent import DDPGAgent3D
env = Env()
#agent = DDPGAgent3D(env)
#actions =  agent.play(10000)

#### Graph NN based agent

In [1]:
from rlvs.molecule_world.env import GraphEnv
from rlvs.agents.ddpg_agent import DDPGAgentGNN
env = GraphEnv()
agent = DDPGAgentGNN(env)
actions =  agent.play(10000)
batch = agent.memory.sample(agent.BATCH_SIZE)

Action: [-10.           9.         -10.           9.05816841  -9.86771488
   8.5309906 ] Reward: -1 E_i: 1 RMSD:  1.1698793585939367
DDPG GraphNN
Instructions for updating:
Use tf.identity instead.


/Users/in-justin.jose/Documents/Projects/E4R/rl-virtual-screening/rlvs/network/actor.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  actor_weights = np.array(self.actor_target.get_weights())
/Users/in-justin.jose/Documents/Projects/E4R/rl-virtual-screening/rlvs/network/actor.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  target_weights = np.array(self.actor.get_weights())
/Users/in-justin.jose/Documents/Projects/E4R/rl-virtual-screening/rlvs/network/critic.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (wh

Episode: 1 Return: -1 episode_length: 1 Max Reward 0 Critic Loss:  47440.03  Actor loss:  219.11685
Action: [-10.          10.          -8.           9.21065617  -8.85055351
   9.93097687] Reward: -1 E_i: 1 RMSD:  1.2019444972321416
DDPG GraphNN


InvalidArgumentError: Matrix size-incompatible: In[0]: [2859,18], In[1]: [6,64] [Op:MatMul]

In [4]:
import numpy as np
import tensorflow as tf
print(len(batch))
xx = [val['state'] for val in batch]

# agent._critiq.action_gradients(xx, agent._actor.actor)

with tf.GradientTape() as tape2:
  action_tensor = agent._actor.actor(xx)

  q = agent._critiq.critic([xx, action_tensor])
  action_loss = -tf.reduce_mean(q)
  action_gradient =  tape2.gradient(action_loss, agent._actor.actor.trainable_variables)         
action_gradient, action_loss.numpy()

1
Instructions for updating:
Use tf.identity instead.


([<tf.Tensor: shape=(18, 64), dtype=float32, numpy=
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
  <tf.Tensor: shape=(18, 64), dtype=float32, numpy=
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
  <tf.Tensor: shape=(18, 64), dtype=float32, numpy=
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
  <tf.Tensor: shape=(18, 64)

In [ ]:
from rlvs.molecule_world.datasets import DataStore
DataStore.init()

xx, xy = DataStore.next()
xx.coords.min(), xx.coords.max()